<span style="float:right">
<a href="http://moldesign.bionano.autodesk.com/" target="_blank" title="About">About</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://forum.bionano.autodesk.com/c/Molecular-Design-Toolkit" target="_blank" title="Forum">Forum</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/autodesk/molecular-design-toolkit/issues" target="_blank" title="Issues">Issues</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="http://bionano.autodesk.com/MolecularDesignToolkit/explore.html" target="_blank" title="Tutorials">Tutorials</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="http://autodesk.github.io/molecular-design-toolkit/" target="_blank" title="Documentation">Documentation</a></span>
</span>
![Molecular Design Toolkit](img/Top.png)
<br>
<center><h1>Example 5: Calculating enthalpic barriers with constraints minimizations </h1> </center>

---

Here, we calculate a torsional rotation barrier of a common amino acid.

 - _Author_: [Aaron Virshup](https://github.com/avirshup), Autodesk Research<br>
 - _Created on_: July 1, 2016
 - _Tags_: quantum chemistry, PES, isomerization, reaction coordinate


In [ ]:
import moldesign as mdt
from moldesign import units as u

%matplotlib notebook
from matplotlib.pyplot import *
try: import seaborn  # optional, makes graphs look better
except ImportError: pass

In [ ]:
mol = mdt.from_name('butane')
mol.draw()

In [ ]:
mol.set_energy_model(mdt.models.GAFF)
mol.energy_model.configure()

In [ ]:
minimization = mol.minimize(nsteps=40)

In [ ]:
minimization.draw()

In [ ]:
bs = mdt.widgets.BondSelector(mol)
bs

In [ ]:
twist = mdt.DihedralMonitor(bs.selected_bonds[0])
twist

In [ ]:
rigid = mdt.Trajectory(mol)
for angle in np.arange(0.0, 360, 5) * u.degree:
    twist.value = angle
    mol.calculate()
    rigid.new_frame(annotation='angle: %s' % twist.value.to(u.degrees))

In [ ]:
rigid.draw()

In [ ]:
figure()
plot(twist(rigid).to(u.degrees), rigid.potential_energy)
xlabel(u'dihedral / º'); ylabel('energy / eV')

In [ ]:
mol.clear_constraints()
mol.positions = rigid.positions[0]

In [ ]:
constraint = twist.constrain()
relaxed = mdt.Trajectory(mol)
for angle in np.arange(0, 360, 5) * u.degree:
    print angle,':',
    
    #add random noise to break symmetry
    mol.positions += np.random.random(mol.positions.shape) * 0.01*u.angstrom
    mol.positions -= mol.center_of_mass
    
    twist.value = angle
    constraint.value = angle
    
    t = mol.minimize(nsteps=100)
    relaxed.new_frame(annotation='angle: %s' % twist.value.to(u.degrees))

In [ ]:
relaxed.draw()

In [ ]:
figure()
x_axis = twist(rigid).to(u.degrees) 
plot(x_axis, rigid.potential_energy, label='rigid')
plot(x_axis, relaxed.potential_energy, label='relaxed')
plot(x_axis, rigid.potential_energy - relaxed.potential_energy, label='rigid - relaxed')
xlabel(u'dihedral / º'); ylabel('energy / eV'); legend()

In [ ]:
from ipywidgets import interact_manual

bs = mdt.widgets.BondSelector(mol)
def show_dihedral():
    figure()
    for bond in bs.selected_bonds:
        dihemon = mdt.DihedralMonitor(bond)
        plot(twist(relaxed).to(u.degrees), dihemon(relaxed).to(u.degrees), label=str(bond))
    legend(); xlabel(u'central twist / º'); ylabel(u'bond twist / º')
interact_manual(show_dihedral)
bs